In [1]:
%cd ..

/home/work/AnacondaProjects/cs7150_project_chyvae


In [2]:
import collections
import torch
import numpy as np
import data_loader.data_loaders as module_data
import model.loss as module_loss
import model.metric as module_metric
import model.model as module_arch
from parse_config import ConfigParser
from trainer import Trainer

In [3]:
%matplotlib inline

In [4]:
# torch.autograd.set_detect_anomaly(True)

In [5]:
# fix random seeds for reproducibility
SEED = 123
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(SEED)

In [6]:
Args = collections.namedtuple('Args', 'config resume device')
config = ConfigParser.from_args(Args(config='config.json', resume=None, device=None))

In [7]:
logger = config.get_logger('train')

In [8]:
# setup data_loader instances
data_loader = config.init_obj('data_loader', module_data)
valid_data_loader = data_loader.split_validation()

Keys in the dataset: KeysView(<numpy.lib.npyio.NpzFile object at 0x7fa776ab7af0>)
Dataset loaded : OK.


In [9]:
# build model architecture, then print to console
model = config.init_obj('arch', module_arch)

In [10]:
criterion = getattr(module_loss, config['loss'])

In [11]:
metrics = [getattr(module_metric, metric) for metric in config['metrics']]

In [12]:
trainable_params = filter(lambda p: p.requires_grad, model.parameters())
optimizer = config.init_obj('optimizer', torch.optim, trainable_params)

lr_scheduler = config.init_obj('lr_scheduler', torch.optim.lr_scheduler, optimizer)

In [13]:
trainer = Trainer(model, criterion, metrics, optimizer, config=config,
                  data_loader=data_loader,
                  valid_data_loader=valid_data_loader,
                  lr_scheduler=lr_scheduler)

In [14]:
trainer._resume_checkpoint('saved/models/Shapes_ChyVAE/0802_175221/checkpoint-epoch100.pth')

Loading checkpoint: saved/models/Shapes_ChyVAE/0802_175221/checkpoint-epoch100.pth ...
Checkpoint loaded. Resume training from epoch 101


In [15]:
trainer.validate()

    epoch          : 0
    loss           : -11122.96253797743
    log_likelihood : -11330.8779296875
    reconstruction_error: 10.866299629211426
